In [1]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from scipy import sparse
import pandas as pd
import numpy as np

In [2]:
model = 'clean_embed_all-mpnet-base-v2.csv'
df = pd.read_csv('/global/home/hpc3552/conversation_analytics/proj/data/banking77/{}'.format(model))
df_orig = pd.read_csv('clean.csv')

In [3]:
#df = df.head(500)
#df_orig = df_orig.head(500)

In [4]:
df_orig=df_orig.to_numpy()

In [5]:
A =  df.drop(['category'], axis=1).to_numpy()
A_sparse = sparse.csr_matrix(A)

sim  = cosine_similarity(A_sparse)
dist = euclidean_distances(A_sparse)

In [6]:
np.argwhere(df_orig[:,0] == "how do i get a new card")

array([[8062]])

In [7]:
print(df_orig[1801,])
print(df_orig[8062,])

['how to get new card?' 'contactless_not_working']
['how do i get a new card' 'order_physical_card']


In [9]:
print(sim[1801,8062])
print(dist[1801,8062])

0.9489806309165698
0.3194350677397905


In [10]:
res = []
for pair in np.argwhere(sim > 0.85):
    i, j = pair
    if j <= i:
        continue
    if df_orig[i, 1] == df_orig[j, 1]:
        continue 
    #print("\ni={}, j={}, sim={:.4f}".format(i, j, sim[i, j]))
    #print(df_orig[i,:])
    #print(df_orig[j,:])
    res.append({
        'sim': sim[i, j],
        'dist': dist[i, j],
        'i': i,
        'message i': df_orig[i,0],
        'label i': df_orig[i,1],
        'j': j,
        'message j': df_orig[j,0],
        'label j': df_orig[j,1],
        
    })
    
results = pd.DataFrame(res)
results = results.sort_values('sim', ascending=False)

In [11]:
results

,sim,dist,i,message i,label i,j,message j,label j
1739,0.979861,0.200694,7029,Why can I not do a transfer,beneficiary_not_allowed,7476,Why can I not make a transfer?,failed_transfer
1274,0.978884,0.205503,3264,It looks like my top-up was cancelled,top_up_reverted,8597,Looks like my top-up has been cancelled,top_up_failed
1316,0.976023,0.218986,3553,There is a payment I don't recognize,card_payment_not_recognised,4792,There is a payment I don't recognise!,direct_debit_payment_not_recognised
1200,0.974622,0.225289,3171,Seems like my top-up has been cancelled,top_up_reverted,8597,Looks like my top-up has been cancelled,top_up_failed
1651,0.973868,0.228611,6067,How long does a transfer take?,pending_transfer,8671,How long does transfers take?,balance_not_updated_after_bank_transfer
...,...,...,...,...,...,...,...,...
1242,0.850079,0.547577,3198,My money is gone. I had topped up recently and...,top_up_reverted,4247,I tried to top up and suddenly my money disapp...,topping_up_by_card
797,0.850055,0.547622,2074,I'm not certain my top-up went through yet.,pending_top_up,8471,"My top-up hasn't gone through, what happened?",top_up_failed
1654,0.850040,0.547650,6067,How long does a transfer take?,pending_transfer,8742,what amount of time does it take for a transfe...,balance_not_updated_after_bank_transfer
1682,0.850022,0.547682,6629,How many days does it take for a transfer to g...,transfer_timing,8742,what amount of time does it take for a transfe...,balance_not_updated_after_bank_transfer


In [12]:
results.to_csv('sim_results.csv', index=False)

In [14]:
pd.set_option("display.max_rows", None)
results[['label i', 'label j']].value_counts()

label i                                           label j                                         
card_arrival                                      card_delivery_estimate                              250
getting_virtual_card                              get_disposable_virtual_card                         230
pending_top_up                                    top_up_failed                                       178
beneficiary_not_allowed                           failed_transfer                                     100
top_up_reverted                                   top_up_failed                                        96
declined_transfer                                 failed_transfer                                      57
pending_top_up                                    top_up_reverted                                      55
transfer_timing                                   balance_not_updated_after_bank_transfer              52
transfer_not_received_by_recipient                bal

In [15]:
res2 = []
for pair in np.argwhere(dist < 0.5):
    i, j = pair
    if j <= i:
        continue
    if df_orig[i, 1] == df_orig[j, 1]:
        continue 
    #print("\ni={}, j={}, sim={:.4f}".format(i, j, sim[i, j]))
    #print(df_orig[i,:])
    #print(df_orig[j,:])
    res2.append({
        'sim': sim[i, j],
        'dist': dist[i, j],
        'i': i,
        'message i': df_orig[i,0],
        'label i': df_orig[i,1],
        'j': j,
        'message j': df_orig[j,0],
        'label j': df_orig[j,1],
        
    })
    
results2 = pd.DataFrame(res2)
results2 = results2.sort_values('dist', ascending=True)

In [16]:
results2

,sim,dist,i,message i,label i,j,message j,label j
783,0.979861,0.200694,7029,Why can I not do a transfer,beneficiary_not_allowed,7476,Why can I not make a transfer?,failed_transfer
582,0.978884,0.205503,3264,It looks like my top-up was cancelled,top_up_reverted,8597,Looks like my top-up has been cancelled,top_up_failed
599,0.976023,0.218986,3553,There is a payment I don't recognize,card_payment_not_recognised,4792,There is a payment I don't recognise!,direct_debit_payment_not_recognised
545,0.974622,0.225289,3171,Seems like my top-up has been cancelled,top_up_reverted,8597,Looks like my top-up has been cancelled,top_up_failed
750,0.973868,0.228611,6067,How long does a transfer take?,pending_transfer,8671,How long does transfers take?,balance_not_updated_after_bank_transfer
762,0.973787,0.228968,6643,How long does it take for a transfer to show u...,transfer_timing,8665,How long do transfers take to show up in my ac...,balance_not_updated_after_bank_transfer
169,0.971832,0.237350,1086,How will I get my card?,card_delivery_estimate,8095,How do I get my card?,order_physical_card
231,0.970317,0.243653,1731,Is there a way to reset my PIN?,pin_blocked,6956,Can I reset my PIN?,change_pin
70,0.969105,0.248574,103,When will my new card arrive?,card_arrival,1107,When can I expect my new card to get here?,card_delivery_estimate
472,0.966882,0.257363,3015,Can i have a virtual card?,getting_virtual_card,8422,can i get a virtual card,get_disposable_virtual_card


In [17]:
results2.to_csv('sim_results2.csv', index=False)

In [18]:
results2['label i'].value_counts()

card_arrival                                        128
pending_top_up                                      122
getting_virtual_card                                 93
top_up_reverted                                      52
transfer_not_received_by_recipient                   48
declined_transfer                                    39
pending_transfer                                     29
beneficiary_not_allowed                              28
transfer_timing                                      23
lost_or_stolen_card                                  19
card_not_working                                     19
pin_blocked                                          18
getting_spare_card                                   17
declined_card_payment                                17
top_up_by_bank_transfer_charge                       17
why_verify_identity                                  14
contactless_not_working                              14
card_linking                                    

In [19]:
results2['label j'].value_counts()

top_up_failed                                       133
card_delivery_estimate                              125
get_disposable_virtual_card                         105
balance_not_updated_after_bank_transfer              68
failed_transfer                                      56
top_up_reverted                                      36
declined_card_payment                                34
activate_my_card                                     25
verify_my_identity                                   24
order_physical_card                                  23
pending_transfer                                     22
reverted_card_payment?                               18
get_physical_card                                    15
transfer_timing                                      14
direct_debit_payment_not_recognised                  13
top_up_by_card_charge                                11
lost_or_stolen_phone                                 11
wrong_exchange_rate_for_cash_withdrawal         

In [20]:
pd.set_option("display.max_rows", None)
results2[['label i', 'label j']].value_counts()

label i                                           label j                                         
card_arrival                                      card_delivery_estimate                              125
getting_virtual_card                              get_disposable_virtual_card                          92
pending_top_up                                    top_up_failed                                        85
top_up_reverted                                   top_up_failed                                        48
pending_top_up                                    top_up_reverted                                      36
declined_transfer                                 failed_transfer                                      27
transfer_not_received_by_recipient                balance_not_updated_after_bank_transfer              26
beneficiary_not_allowed                           failed_transfer                                      26
transfer_timing                                   bal